In [ ]:
!pip install scikit-plot

In [ ]:
# Load libraries
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn import preprocessing
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, auc
from keras.layers import Dense, Dropout, Activation, Embedding, Flatten, LSTM
from sklearn.metrics import plot_confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt
import scikitplot as skplt
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.preprocessing import label_binarize
from scipy import interp
from itertools import cycle
from sklearn.model_selection import KFold,cross_val_score
kfold=KFold(5,True,3)
cvscores = []
import h5py

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
 # data is being refined, Here we fist get our data to be converted into pandas dataframae
# then we drop empty spaces and eradicate useless indices and getting data in float type
def cleaningdata(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)
# fetching dataset from path
dataset = pd.read_csv('/content/drive/MyDrive/Code/dataset/IoT-BoT.csv', encoding='utf-8')
# for object type data we will label it and transform into apporopriate type fo data after using fit_transform on that colomn to avoid errors
for column in dataset.columns:
    if dataset[column].dtype == type(object):
        le = LabelEncoder()
        dataset[column] = le.fit_transform(dataset[column])
ds = cleaningdata(dataset)
X = ds.iloc[:, 0:85].values
# collecting our LABEL colomn in y
y = ds.iloc[:, -2].values


In [ ]:
#### BLSTM #######
def cleaningdata(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)
# fetching dataset from path
dataset = pd.read_csv('/content/drive/My Drive/Code/dataset/IoT Intrusion Dataset 2020.csv', encoding='utf-8')
# for object type data we will label it and transform into apporopriate type fo data after using fit_transform on that colomn to avoid errors
for column in dataset.columns:
    if dataset[column].dtype == type(object):
        le = LabelEncoder()
        dataset[column] = le.fit_transform(dataset[column])
data = cleaningdata(dataset)

# collecting all required colomns in data to X
X = data.iloc[:, 0:85].values

# collecting our LABEL colomn in y
y = data.iloc[:, -2].values


In [ ]:
for train, test in kfold.split(X,y):
   X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=52)
   from keras.utils.np_utils import to_categorical
   sc = StandardScaler()
   X_train = sc.fit_transform(X_train)
   X_test = sc.transform(X_test)

   X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
   X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

   x_train = np.array(X_train)
   x_test = np.array(X_test)
   y_train1 = np.array(y_train)
   y_test1 = np.array(y_test)

   y_train= to_categorical(y_train1)
   y_test= to_categorical(y_test1)
# 1. define the network
   from keras.layers import Bidirectional
   model = Sequential()
   model.add(Bidirectional(LSTM(80, input_dim=85, return_sequences=True)))
   model.add(Dropout(0.1))
   model.add(Bidirectional(LSTM(40)))
   model.add(Dropout(0.1))
   model.add(Dense(128, activation='relu'))
   model.add(Dropout(0.1))
   model.add(Dense(5, activation='softmax'))
   model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy','mae'])
   history = model.fit(X_train, y_train, epochs = 15, batch_size=64, verbose=1,shuffle=False)
   scores = model.evaluate(X_test, y_test, verbose=1)
   print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
   cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))


Epoch 1/15
7818/7818 [==============================] - 54s 7ms/step - loss: 0.0114 - accuracy: 0.9969 - mae: 0.0025
Epoch 2/15
7818/7818 [==============================] - 56s 7ms/step - loss: 8.4819e-04 - accuracy: 0.9998 - mae: 1.2615e-04
Epoch 3/15
7818/7818 [==============================] - 56s 7ms/step - loss: 5.6785e-04 - accuracy: 0.9998 - mae: 8.3241e-05
Epoch 4/15
7818/7818 [==============================] - 57s 7ms/step - loss: 5.1402e-04 - accuracy: 0.9999 - mae: 7.4885e-05
Epoch 5/15
7818/7818 [==============================] - 56s 7ms/step - loss: 4.0382e-04 - accuracy: 0.9999 - mae: 5.7162e-05
Epoch 6/15
7818/7818 [==============================] - 56s 7ms/step - loss: 3.7946e-04 - accuracy: 0.9999 - mae: 5.0787e-05
Epoch 7/15
7818/7818 [==============================] - 56s 7ms/step - loss: 3.0643e-04 - accuracy: 0.9999 - mae: 4.4155e-05
Epoch 8/15
7818/7818 [==============================] - 58s 7ms/step - loss: 2.5443e-04 - accuracy: 0.9999 - mae: 3.5023e-05
Epoch 9/

In [ ]:
target_names = ['DoS', 'DDOS', 'Reconnaissance', 'Normal','Theft']

In [ ]:
y_pred_test = model.predict(X_test)
# print test_scores of accuracy,precision,recall,f1-score
print("Test_Accuracy:",metrics.accuracy_score(y_test, np.round(y_pred_test)))
print("Test_Precision:",metrics.precision_score(y_test, np.round(y_pred_test), average='weighted'))
print("Test_Recall:",metrics.recall_score(y_test, np.round(y_pred_test), average='weighted'))
print("Test_f1-score:",metrics.f1_score(y_test, np.round(y_pred_test), average='weighted'))
print(classification_report(y_test, np.round(y_pred_test), target_names=target_names))

# print train_scores of accuracy,precision,recall,f1-score
y_pred_train = model.predict(X_train)
print("Train_Accuracy:",metrics.accuracy_score(y_train, np.round(y_pred_train)))
print("Train_Precision:",metrics.precision_score(y_train, np.round(y_pred_train), average='weighted'))
print("Train_Recall:",metrics.recall_score(y_train, np.round(y_pred_train), average='weighted'))
print("Train_f1-score:",metrics.f1_score(y_train, np.round(y_pred_train), average='weighted'))
print(classification_report(y_train, np.round(y_pred_train), target_names=target_names))

Test_Accuracy: 0.9999120583932269
Test_Precision: 0.9999120697742933
Test_Recall: 0.9999120583932269
Test_f1-score: 0.999912059304131
                precision    recall  f1-score   support

           DoS       1.00      1.00      1.00     11815
          DDOS       1.00      1.00      1.00      7109
Reconnaissance       1.00      1.00      1.00     83057
        Normal       1.00      1.00      1.00      8060
         Theft       1.00      1.00      1.00     15042

     micro avg       1.00      1.00      1.00    125083
     macro avg       1.00      1.00      1.00    125083
  weighted avg       1.00      1.00      1.00    125083
   samples avg       1.00      1.00      1.00    125083

Train_Accuracy: 0.9999760159254255
Train_Precision: 0.999976016867048
Train_Recall: 0.9999760159254255
Train_f1-score: 0.9999760160333039
                precision    recall  f1-score   support

           DoS       1.00      1.00      1.00     47576
          DDOS       1.00      1.00      1.00     28